# Understanding ```sympyfy```: An Analytical Momentum-Space Hamiltonian Expresser.

In this notebook, we shall learn the intricacies that go behind the ```sympyfy``` function. But first, let us dive into the motivation behind writing such a function.

The ```sympyfy``` function makes great use of the tight-binding (TB) model. In a nutshell, the TB model is used in Condensed Matter Physics to study the properties of the band structures of different kinds of conductors, semiconductors, and insulators. In particular, we care about how electrons move and interact withing the crystal lattice site framework by considering how they interact with other electrons within nearby lattices. To analyze these interactions, the wavefunctions of the involved electrons should capture their locality. Therefore, we need to choose a set of wavefunctions that can help us characterize the behavior of electrons within the local landscape while neglecting far-off electron interactions. Luckily, there exist a set of localized wavefunctions that can help us do so - Wannier functions. The scope of Wannier functions will be limited in this tutorial. Using these wavefunctions, we can characterize the behavior of electrons across nearby lattice sites as the wave functions of the electrons involved will overlap which will lead to the hopping of electrons from one orbital to another (inter-atomic orbital hopping). These orbital hoppings can be within the same unit cell or across different unit cells and is the main phenomenon that is captured by ```sympyfy```.

The TB model is useful when applied to materials whose electrons are weakly correlated. Like many other models, the TB model has its own Hamiltonian equation. This equation has 2 significant terms, one of which is the hopping term which describes the hopping of electrons between local neighboring sites. This is the term that ```sympyfy``` captures. The TB model captures valuable insights on the electronic properties of materials.


The Hamiltonian of the TB model looks like this:
$$
\hat{H} = \sum_{\vec{R}\vec{\delta}}| \vec{R}> \cal{t} <\vec{R} + \vec{\delta}| + \sum_{\vec{R}} |\vec{R}>U<\vec{R}|
$$

where $\vec{\delta}$ is a set of vectors pointing from $\vec{R}$ to its nearest neighbors.

As mentioned before, the TB model is particularly useful in figuring out the band structure properties of different kinds of materials. Condensed matter experimental physicists, as well as theoretical physicists, interface with the TB model in quite different ways. Experimental physicists can interface with the TB model when performing electronic band structure measurements such as Angle-Resolved Photoemission Spectroscopy (ARPES). They need to know whether their analyses of their experimental data fits with what the theory dictates. Moreover, the TB model is easily tractable, mathematically, which allows for an easy-to-view analytical (Hamiltonian) solution for whatever weakly-correlated system one is analyzing. This tractability proves efficient to experimentalists as it can save time for them to quickly check whether their experimental data makes sense.
There is one issue however, though the analytical solution to the TB model is useful, it can end up being verbose and drawn-out which is counter to what an experimentalist would like. To drive the point home, an experimentalist would like a succinct, compact analytical expression of the compound they are working with. Therefore, it does them no good to look at a length (Hamiltonian) expression. On top of this, condensed matter physicists are often interested in looking at systems at or near the Fermi level as this is where interesting physics happens such as when electrons cross from the valence band to the conduction band which is interesting when we look at (super)conductivity and electrical properties.

Therefore, there is a need to develop a function that can provide a reduced analytical Hamiltonian expression of whatever element that is analyzed. This is what ```sympyfy``` does! Let us look at the function definition:

```def sympyfy(tb_lat_obj, analytical = True, precision = 6):```

The ```tb_lat_obj``` is a TRIQS TBLattice Object (Tight-Binding Lattice Object) which is the only mandatory parameter in our function. To obtain the ```tb_lat_obj```, TRIQS fortunately provides functions from which you can provide your model to a TRIQS TBLattice object. These functions are:
1. ```TB_from_pythTB``` which converts your model from a pythTB model to a TRIQS TBLattice Object.
2. ```TB_from_wannier90``` which converts your model from a Wannier90 model to a TRIQS TBLattice Object. It reads wannier90 output and convert it to a TBLattice object.

This makes the ```tb_lat_obj``` to be the only parameter that interfaces with other TRIQS package functions.
At this time, it is recommended that one use the ```TB_from_pythTB``` as you can easily initiate cutoffs to your pythTB model before converting it to a TBLattice object. This allows one to be more in control of their model and look at the behavior of the output at different cutoffs. Such cutoff parameters, possible with the pythTB includes the:
1. ```zero_energy```: sets the zero of the energy in the band structure, which is almost always set at the Fermi level.
2. ```min_hopping_norm```: the hopping terms read from Wannier90. Hopping amplitudes (measured in electron-volts) less than the set ```min_hopping_norm``` will not be included in the resulting calculations.
3. ```max_distance```: Any hopping distance greater than the set ```max_distance``` will be ignored.

The ```analytical``` flag ensures that the default Hamiltonian that is going to be returned once ```sympyfy``` is run is going to be analytical. The user can change the ```analytical``` flag to ```False``` to see the resulting numerical Hamiltonian. The numerical form of the Hamiltonian only depends on the k-space vector components, i.e., the lattice constants and lattice vectors are expressed numerically.

Finally, the ```precision``` parameter is there if the user wants a more compact Hamiltonian form. By default, the floating point values from Wannier90 are in the order of $10^{-6}$. While this is quite precise, it can be difficult to notice similar hopping amplitude values that may arise due to symmetrical hops in opposite directions.

## Calling the function ##

Here is an example on how the function can be called on the model of a Strontium compound - $SrCaCu_{2}O_4$.

In [1]:
from triqs.lattice.utils import TB_from_pythTB
w90_input = w90('SrCaCu2O4 Files', 'sccuo2')
fermi_ev = 9.6082
w90_model = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.05, max_distance = None)
w90_triqs = TB_from_pythTB(w90_model)

NameError: name 'w90' is not defined

### Calling ```sympyfy``` in different scenarios ###

In [2]:
# calling the functions in different scenarios
print("The analytical expression \n", sympyfy(w90_triqs))
print("The numerical expression \n", sympyfy(w90_triqs, False))
print("The more precise numerical expression \n", sympyfy(w90_triqs, False, 3))

NameError: name 'sympyfy' is not defined

Here are the results of the scenarios above:

The analytical expression 
 $$$[
  [-1.002632*cos(a3k) - 0.124416*cos(2*a3k) + 0.018272
  -0.056436*I*sin(a1k) - 0.056434*I*sin(a1k - a3k) + 0.056436*cos(a1k) + 0.056434*cos(a1k - a3k)]
 
 [0.056436*I*sin(a1k) + 0.056434*I*sin(a1k - a3k) + 0.056436*cos(a1k) + 0.056434*cos(a1k - a3k)
  -0.904748*cos(a3k) - 0.128046*cos(2*a3k) - 0.130246]
  ]$$$


The numerical expression 
 $$$[
  [-1.002632*cos(3.939405*kz) - 0.124416*cos(7.87881*kz) + 0.018272
  -0.056436*I*sin(1.76207*kx + 1.76207*ky) - 0.056434*I*sin(1.76207*kx + 1.76207*ky - 3.939405*kz) + 0.056436*cos(1.76207*kx + 1.76207*ky) + 0.056434*cos(1.76207*kx + 1.76207*ky - 3.939405*kz)]
 
 [0.056436*I*sin(1.76207*kx + 1.76207*ky) + 0.056434*I*sin(1.76207*kx + 1.76207*ky - 3.939405*kz) + 0.056436*cos(1.76207*kx + 1.76207*ky) + 0.056434*cos(1.76207*kx + 1.76207*ky - 3.939405*kz)
  -0.904748*cos(3.939405*kz) - 0.128046*cos(7.87881*kz) - 0.130246]
  ]$$$


The more precise numerical expression 
 $$$[
  [-1.002*cos(3.939*kz) - 0.124*cos(7.878*kz) + 0.018
  -0.056*I*sin(1.762*kx + 1.762*ky) - 0.056*I*sin(1.762*kx + 1.762*ky - 3.939*kz) + 0.056*cos(1.762*kx + 1.762*ky) + 0.056*cos(1.762*kx + 1.762*ky - 3.939*kz)]
 [0.056*I*sin(1.762*kx + 1.762*ky) + 0.056*I*sin(1.762*kx + 1.762*ky - 3.939*kz) + 0.056*cos(1.762*kx + 1.762*ky) + 0.056*cos(1.762*kx + 1.762*ky - 3.939*kz)
  -0.904*cos(3.939*kz) - 0.128*cos(7.878*kz) - 0.13]
  ]$$$